In [192]:
for i, j in property_dict.items():
    dt_tag = room_item.find('dt', text=j)
    if dt_tag:
        dd_tag = dt_tag.find_next('dd')
        if dd_tag:
            temp_property_data[i] = dd_tag.text.strip()
        else:
            temp_property_data[i] = ''
    else:
        temp_property_data[i] = ''

In [193]:
temp_property_data

{'price': '3,080万円',
 'transportation': '東京メトロ丸ノ内線 新宿三丁目駅 徒歩9分\r\n  \r\n  東京メトロ丸ノ内線 新宿駅 徒歩14分\r\n  \r\n  \r\n  JR山手線 新宿駅 徒歩14分',
 'address': '東京都新宿区新宿',
 'year_built': '1983年8月（築39年）',
 'building_structure': 'SRC造2階/9階建 総戸数112戸',
 'exclusive_area': '29.11m²\r\n（バルコニー 4.53m²）',
 'floor_plan': '1R バストイレ別',
 'transaction_method': '仲介',
 'delivery': '相談',
 'current_condition': '空室',
 'cap_rate': '3.89％',
 'full_occupancy_incom': '120万円 / 10万円\r\n情報の見方',
 'property_name': '東京都新宿区 投資用マンション',
 'land_rights': '所有権',
 'management_fee_repair_reserve_fund': '7,900円 / 7,900円',
 'management_company': '',
 'management_method': '全部委託\r\n  \r\n    /\r\n    日勤',
 'last_update_date': '',
 'scheduled_update_date': '2023年 6月 5日',
 'management_id': ''}

In [190]:
#!/usr/bin/env python
# cofing: utf-8

# from retry import retry
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
from tqdm import tqdm
import re
import math
# from logging import getLogger, StreamHandler, Formatter, FileHandler, DEBUG
import yaml
import os
from retry import retry
import sys
import time

notebook = True
if notebook:
    work_dir = '/Users/satomitakei/property_valuation_calculator'
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = 'tokyo'
else:
    work_dir = os.getcwd()
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = sys.argv[1]
if area_name == 'tokyo':
    base_url = config['base_url_tokyo']
elif area_name == 'osaka':
    base_url = config['base_url_osaka']
elif area_name == 'fukuoka':
    base_url = config['base_url_fukuoka']

def write_log(log_file, text):
    f = open(log_file, 'a', encoding='UTF-8')
    # f.write(text)
    # f.close()
    print(text)

diff_jst_from_utc = 0
start_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
now_time = (dt.datetime.now() +
            dt.timedelta(hours=diff_jst_from_utc)).strftime('%Y%m%d_%H%M')

log_dir = work_dir + f'/log/scraping'
os.makedirs(log_dir, exist_ok=True)
log_file = log_dir + f'/{now_time}_log.txt'
f = open(log_file, 'w', encoding='UTF-8')
f.close()

text = 'processing_start_time:' + str(start_time.replace(microsecond=0)) + '\n'
write_log(log_file, text)
excution_date = dt.datetime.today().strftime('%Y%m%d')

# file_name = 'suumo_baibai'
# excution_date = dt.datetime.today().strftime('%Y%m%d')

def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup
# @retry(tries=3, delay=10, backoff=2)
# def main():
all_data = []

# 基本ページurl 
page = '1'
url = base_url.format(page = page)
write_log(log_file,'base_url:'+url)
# get html
item = get_html(url)

# extract all items
total_rooms = int(re.sub(r"\D", "",item.find(True,"strong", class_="result_num").get_text()))
max_page = math.floor(total_rooms/50)+ 1
text = f"max_page:{max_page} \n"
write_log(log_file,text)

url_list = []
all_data = []
error_page = []
# 物件URLの取得
data= {}

def extract_value(pattern,temp_property):
    match = re.search(pattern, temp_property)
    if match:
        return match.group(1)

# for i in tqdm(range(max_page+1)): 
for i in tqdm(range(2)): 
    url = base_url.format(page = str(i))
    # get html
    item = get_html(url)
    for j in item.findAll(href=re.compile("pp1/s/.*/re_.*/")):
        room_url = 'https://www.kenbiya.com/'+j.get('href')
        room_item = get_html(room_url)
        property_dict = {
            'price': '価格',
            'transportation': '交通',
            'address': '住所',
            'year_built': '築年月',
            'building_structure': '建物構造/階数',
            'exclusive_area': '専有面積',
            'floor_plan': '間取り',
            'transaction_method': '取引態様',
            'delivery': '引渡',
            'current_condition': '現況',
            'cap_rate': '満室時利回り',
            'full_occupancy_incom': '満室時年収/月収',
            'property_name': '物件名',
            'land_rights': '土地権利',
            'management_fee_repair_reserve_fund': '管理費/修繕積立',
            'management_company': '管理会社',
            'management_method': '管理方式/管理人',
            'last_update_date': '直前の更新日',
            'scheduled_update_date': '更新予定日',
            'management_id': '管理ID'
        }
        temp_property_data={}
        for i, j in property_dict.items():
            dt_tag = room_item.find('dt', text=j)
            if dt_tag:
                dd_tag = dt_tag.find_next('dd')
                if dd_tag:
                    temp_property_data[i] = dd_tag.text.strip()
                else:
                    temp_property_data[i] = ''
            else:
                temp_property_data[i] = ''
        try:
            # 物件詳細のデータを収集
            # マンション名
            data["property_name"] = temp_property_data['property_name']

            # 価格
            data["price"] = int(temp_property_data['price'].replace(',', '').replace('万円', '')) * 10000

            # 交通
            data["train_line"] = temp_property_data['transportation'].split()[0]
            data["station"] = temp_property_data['transportation'].split()[1]
            temp_minutes_from_station = temp_property_data['transportation'].split()[2]
            pattern = r'徒歩(\d.+)分'
            data["minutes_from_station"] = extract_value(pattern,temp_minutes_from_station)

            # 住所
            temp_address= temp_property_data['address']
            pattern = r'^(.+(都|道|府|県))'
            data["prefecture_name"] = extract_value(pattern,temp_address)
            pattern = r'^(.+(市|区))'
            data["city_name"] = extract_value(pattern,temp_address)
            data["town_name"] = temp_address.replace(data["prefecture_name"],'').replace(data["city_name"],'')

            # 築年数
            temp_year_built = temp_property_data['year_built']
            pattern = r'（築(\d.+)年）'
            data["year_built"] = int(extract_value(pattern,temp_year_built))

            # 構造
            temp_building_structure= temp_property_data['building_structure']
            pattern = r'^(.+)造'
            data["structure"] = extract_value(pattern,temp_building_structure)

            pattern = r'^.+造(\d+)階'
            data["floor"] = extract_value(pattern,temp_building_structure)

            pattern = r'^.+(\d.+)階建'
            data["max_floor"] = extract_value(pattern,temp_building_structure)

            # 総戸数
            temp_total_rooms = temp_property_data['building_structure'].strip()
            pattern = r'総戸数(\d+)戸'
            data["total_rooms"] = extract_value(pattern,temp_total_rooms)

            # 専有面積
            temp_exclusive_area= temp_property_data['exclusive_area']
            pattern = r'^(\d.+)m²'
            data["exclusive_area"] = extract_value(pattern,temp_exclusive_area)

            # 間取り
            data["floor_plan"] = temp_property_data['floor_plan'].split()[0]

            # 方角
            data["direction"] = temp_property_data['floor_plan'].split()[1]

            # 取引態様
            data["transaction_method"] = temp_property_data['transaction_method']

            # 引渡
            data["delivery"] = temp_property_data['delivery']

            # 現況
            data["current_condition"] = temp_property_data['current_condition']

            # 満室時利回り
            data["cap_rate"] = float(temp_property_data['cap_rate'].replace('％','')) / 100

            # 満室時年収
            data["full_occupancy_incom"] = int(float(temp_property_data['full_occupancy_incom'].split()[0].replace('万円','')) * 10000)

            # 土地権利
            data["land_rights"] = temp_property_data['land_rights']

            # 'management_fee_repair_reserve_fund': '管理費/修繕積立',
            data["management_fee"] = int(temp_property_data['management_fee_repair_reserve_fund'].split('/')[0].replace('円','').replace(',',''))
            data["repair_reserve_fund"] = int(temp_property_data['management_fee_repair_reserve_fund'].split('/')[1].replace('円','').replace(',',''))

            # 管理会社
            data["management_company"] = temp_property_data['management_company']

            # 'management_method': '管理方式/管理人',
            data["management_method"] = re.findall('(.*)\r\n  \r\n*', temp_property_data['management_method'].split('/')[0])[0]
            data["management_person"] = re.findall('\r\n    (.*)', temp_property_data['management_method'].split('/')[1])[0]

            # 直前の更新日
            date_format = '%Y年%m月%d日'
            text = temp_property_data['last_update_date'].replace(' ','')
            data["last_update_date"] = datetime.strptime(text, date_format).date()


            # 'scheduled_update_date': '更新予定日',
            text = temp_property_data['scheduled_update_date'].replace(' ','')
            data["scheduled_update_date"] = datetime.strptime(text, date_format).date()

            # 'management_id': '管理ID'
            data["management_id"] = temp_property_data['management_id']
            write_log(log_file,'Done:'+data["property_name"])
            time.sleep(3)
        except:
            time.sleep(3)
            write_log(log_file,'error')
        
        #     
        #     data["exclusive_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[4].getText().strip()
        #     data["other_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[5].getText().strip()
        #     data["stories"] = room_item.findAll("td", {"class": "w290 bdCell"})[6].getText().strip()
        #     data["completion"] = room_item.findAll("td", {"class": "w290 bdCell"})[7].getText().strip()
        #     data["adress"] = room_item.findAll("td", {"class": "w290 bdCell"})[8].getText().strip()
        #     data["access"] = room_item.findAll("td", {"class": "w290 bdCell"})[9].getText().strip()

        #     # 物件詳細概要ページからのデータを取得
        #     room_detail_item = get_html(room_detail_url)
        #     data["move_in_date"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[12].getText().strip()
        #     data["direction"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[15].getText().strip()
        #     data["reform"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[16].getText().strip()
        #     data["ownership"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[23].getText().strip()
        #     data["use_district"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[24].getText().strip()
        #     data["url"] = room_url
        #     data["log_date"] = excution_date
        #     all_data.append(data)
        # except:
        #     error_page.append(room_url)

    df = pd.DataFrame(data)
    df.to_csv(work_dir+f'/scraping_raw/kenbiya_baibai_{excution_date}.csv',index = False)
    
text = 'df_shape:{}\n'.format(df.shape)
write_log(log_file,text)

end_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
text = 'predicting done.\nend_time:{}\n'.format(end_time)
write_log(log_file,text)

processing_time = end_time - start_time
text = 'processing_time:{}\n'.format(processing_time)
write_log(log_file,text)

processing_start_time:2023-03-05 16:30:37

base_url:https://www.kenbiya.com/pp1/s/tokyo/n-1/ctk=292_300_311_299_293_302_307_298_294_301_310_314_295_303_308_313_304_306_309_312_296_305_297/
max_page:159 



 50%|█████     | 1/2 [00:04<00:04,  4.14s/it]


AttributeError: 'NoneType' object has no attribute 'find_next'

In [1]:
#!/usr/bin/env python
# cofing: utf-8

# from retry import retry
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
from tqdm import tqdm
import re
import math
# from logging import getLogger, StreamHandler, Formatter, FileHandler, DEBUG
import yaml
import os
from retry import retry
import sys
import time

notebook = True
if notebook:
    work_dir = '/Users/satomitakei/property_valuation_calculator'
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = 'tokyo'
else:
    work_dir = os.getcwd()
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = sys.argv[1]
if area_name == 'tokyo':
    base_url = config['base_url_tokyo']
elif area_name == 'osaka':
    base_url = config['base_url_osaka']
elif area_name == 'fukuoka':
    base_url = config['base_url_fukuoka']

def write_log(log_file, text):
    f = open(log_file, 'a', encoding='UTF-8')
    # f.write(text)
    # f.close()
    print(text)

diff_jst_from_utc = 0
start_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
now_time = (dt.datetime.now() +
            dt.timedelta(hours=diff_jst_from_utc)).strftime('%Y%m%d_%H%M')

log_dir = work_dir + f'/log/scraping'
os.makedirs(log_dir, exist_ok=True)
log_file = log_dir + f'/{now_time}_log.txt'
f = open(log_file, 'w', encoding='UTF-8')
f.close()

text = 'processing_start_time:' + str(start_time.replace(microsecond=0)) + '\n'
write_log(log_file, text)
excution_date = dt.datetime.today().strftime('%Y%m%d')

# file_name = 'suumo_baibai'
# excution_date = dt.datetime.today().strftime('%Y%m%d')

def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup
@retry(tries=3, delay=10, backoff=2)
def main():
    all_data = []

    # 基本ページurl 
    page = '1'
    url = base_url.format(page = page)
    write_log(log_file,'base_url:'+url)
    # get html
    item = get_html(url)

    # extract all items
    total_rooms = int(re.sub(r"\D", "",item.find(True,"strong", class_="result_num").get_text()))
    max_page = math.floor(total_rooms/50)+ 1
    text = f"max_page:{max_page} \n"
    write_log(log_file,text)

    url_list = []
    all_data = []
    error_page = []
    # 物件URLの取得
    data= {}
    # for i in tqdm(range(max_page+1)): 
    for i in tqdm(range(2)): 
        url = base_url.format(page = str(i))
        # get html
        item = get_html(url)
        for j in item.findAll(href=re.compile("pp1/s/.*/re_.*/")):
            print(j.get('href'))
            room_url = 'https://www.kenbiya.com/'+j.get('href')
            write_log(log_file,'room_url'+room_url)
            print(room_url)
            try:
                room_item = get_html(room_url)
                # 物件詳細のデータを収集
                print(room_item)
                data["mansion_name"] = room_item.find("h1", {"class": "inner ttl"}).getText().strip()
                print('mansion_name:'+data["mansion_name"])
                write_log(log_file,data["mansion_name"])
                time.sleep(1)
                print('room done')
            except:
                write_log(log_file,'error')
            #     data["price"] = room_item.find("p", {"class": "mt7 b"}).getText().strip()
            #     data["floor_plan"] = room_item.findAll("td", {"class": "w290 bdCell"})[1].getText().strip()
            #     data["total_rooms"] = room_item.findAll("td", {"class": "w290 bdCell"})[3].getText().strip()
            #     data["exclusive_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[4].getText().strip()
            #     data["other_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[5].getText().strip()
            #     data["stories"] = room_item.findAll("td", {"class": "w290 bdCell"})[6].getText().strip()
            #     data["completion"] = room_item.findAll("td", {"class": "w290 bdCell"})[7].getText().strip()
            #     data["adress"] = room_item.findAll("td", {"class": "w290 bdCell"})[8].getText().strip()
            #     data["access"] = room_item.findAll("td", {"class": "w290 bdCell"})[9].getText().strip()

            #     # 物件詳細概要ページからのデータを取得
            #     room_detail_item = get_html(room_detail_url)
            #     data["move_in_date"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[12].getText().strip()
            #     data["direction"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[15].getText().strip()
            #     data["reform"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[16].getText().strip()
            #     data["ownership"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[23].getText().strip()
            #     data["use_district"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[24].getText().strip()
            #     data["url"] = room_url
            #     data["log_date"] = excution_date
            #     all_data.append(data)
            # except:
            #     error_page.append(room_url)

        df = pd.DataFrame(data)
        df.to_csv(work_dir+f'/scraping_raw/kenbiya_baibai_{excution_date}.csv',index = False)
        
    text = 'df_shape:{}\n'.format(df.shape)
    write_log(log_file,text)

    end_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
    text = 'predicting done.\nend_time:{}\n'.format(end_time)
    write_log(log_file,text)

    processing_time = end_time - start_time
    text = 'processing_time:{}\n'.format(processing_time)
    write_log(log_file,text)

if __name__ == '__main__':
    main()


processing_start_time:2023-03-05 11:22:01

base_url:https://www.kenbiya.com/pp1/s/tokyo/n-1/ctk=292_300_311_299_293_302_307_298_294_301_310_314_295_303_308_313_304_306_309_312_296_305_297/
max_page:159 



  0%|          | 0/2 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


/pp1/s/tokyo/minato-ku/re_29664374di/
room_urlhttps://www.kenbiya.com//pp1/s/tokyo/minato-ku/re_29664374di/
https://www.kenbiya.com//pp1/s/tokyo/minato-ku/re_29664374di/
<!DOCTYPE html>

<html lang="ja">
<head>
<meta charset="utf-8"/>
<title>3駅利用可！港区白赤坂8丁目アドレス！2004年築！（No.29664374di）｜健美家</title>
<meta content="ビーエフエステートが掲載する3駅利用可！港区白赤坂8丁目アドレス！2004年築！（No.29664374di）の物件詳細ページです。不動産投資と収益物件の総合サイト健美家では、似た条件の物件や価格別、利回りが高い順、新着順などの条件で簡単に探せます。" name="description"/>
<meta content="不動産投資,東京都,港区,3駅利用可！港区白赤坂8丁目アドレス！2004年築！,29664374di,健美家,けんびや" name="keywords"/>
<meta content="noarchive,index,follow" name="robots"/>
<script>
window.dataLayer = window.dataLayer || [];
var ga_uid = ga_getUID();
if (ga_uid.login == 1) {
    dataLayer.push({'loginStatus' : 'Logged-in'});
}
else {
    dataLayer.push({'loginStatus' : 'Not_Logged-in'});
}
dataLayer.push({'userId' : ga_uid.uid});

function ga_getUID() {
    var cookies = document.cookie;
    var ret = {login: 0, uid: ''};
    if( cookies != '' ) {
        var

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<!DOCTYPE html>

<html lang="ja">
<head>
<meta charset="utf-8"/>
<title>表面利回り：約4.28% 年間収入：1,500,000円（No.2966424z2g）｜健美家</title>
<meta content="大成有楽不動産販売 荻窪センターが掲載する表面利回り：約4.28% 年間収入：1,500,000円（No.2966424z2g）の物件詳細ページです。不動産投資と収益物件の総合サイト健美家では、似た条件の物件や価格別、利回りが高い順、新着順などの条件で簡単に探せます。" name="description"/>
<meta content="不動産投資,東京都,中野区,表面利回り：約4.28% 年間収入：1,500,000円,2966424z2g,健美家,けんびや" name="keywords"/>
<meta content="noarchive,index,follow" name="robots"/>
<script>
window.dataLayer = window.dataLayer || [];
var ga_uid = ga_getUID();
if (ga_uid.login == 1) {
    dataLayer.push({'loginStatus' : 'Logged-in'});
}
else {
    dataLayer.push({'loginStatus' : 'Not_Logged-in'});
}
dataLayer.push({'userId' : ga_uid.uid});

function ga_getUID() {
    var cookies = document.cookie;
    var ret = {login: 0, uid: ''};
    if( cookies != '' ) {
        var c = cookies.split( '; ' );
        for( var i=0; i<c.length; i++ ) {
            var cookie = c[i].split( '=' );
	    if (cookie[0] == 'K') {
		ret.uid = c

...

/pp1/s/tokyo/shinagawa-ku/re_29662696bt/
room_urlhttps://www.kenbiya.com//pp1/s/tokyo/shinagawa-ku/re_29662696bt/
https://www.kenbiya.com//pp1/s/tokyo/shinagawa-ku/re_29662696bt/
<!DOCTYPE html>

<html lang="ja">
<head>
<meta charset="utf-8"/>
<title>●３路線利用可●１１階建４階部分●オーナーチェンジ物件（No.29662696bt）｜健美家</title>
<meta content="住友不動産販売 大井町営業センターが掲載する●３路線利用可●１１階建４階部分●オーナーチェンジ物件（No.29662696bt）の物件詳細ページです。不動産投資と収益物件の総合サイト健美家では、似た条件の物件や価格別、利回りが高い順、新着順などの条件で簡単に探せます。" name="description"/>
<meta content="不動産投資,東京都,品川区,●３路線利用可●１１階建４階部分●オーナーチェンジ物件,29662696bt,健美家,けんびや" name="keywords"/>
<meta content="noarchive,index,follow" name="robots"/>
<script>
window.dataLayer = window.dataLayer || [];
var ga_uid = ga_getUID();
if (ga_uid.login == 1) {
    dataLayer.push({'loginStatus' : 'Logged-in'});
}
else {
    dataLayer.push({'loginStatus' : 'Not_Logged-in'});
}
dataLayer.push({'userId' : ga_uid.uid});

function ga_getUID() {
    var cookies = document.cookie;
    var ret = {login: 0, uid: ''};
    if( cookies 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<!DOCTYPE html>

<html lang="ja">
<head>
<meta charset="utf-8"/>
<title>品川区 2980万円 4.22% 投資用マンション（No.29662684mg）｜健美家</title>
<meta content="住友不動産販売 大井町営業センターが掲載する品川区 2980万円 4.22% 投資用マンション（No.29662684mg）の物件詳細ページです。不動産投資と収益物件の総合サイト健美家では、似た条件の物件や価格別、利回りが高い順、新着順などの条件で簡単に探せます。" name="description"/>
<meta content="不動産投資,東京都,品川区,品川区 2980万円 4.22% 投資用マンション,29662684mg,健美家,けんびや" name="keywords"/>
<meta content="noarchive,index,follow" name="robots"/>
<script>
window.dataLayer = window.dataLayer || [];
var ga_uid = ga_getUID();
if (ga_uid.login == 1) {
    dataLayer.push({'loginStatus' : 'Logged-in'});
}
else {
    dataLayer.push({'loginStatus' : 'Not_Logged-in'});
}
dataLayer.push({'userId' : ga_uid.uid});

function ga_getUID() {
    var cookies = document.cookie;
    var ret = {login: 0, uid: ''};
    if( cookies != '' ) {
        var c = cookies.split( '; ' );
        for( var i=0; i<c.length; i++ ) {
            var cookie = c[i].split( '=' );
	    if (cookie[0] == 'K') {
		ret.uid = cookie[1]

100%|██████████| 2/2 [00:05<00:00,  2.53s/it]


error
df_shape:(0, 0)

predicting done.
end_time:2023-03-05 11:22:06.573691

processing_time:0:00:05.373980



In [3]:
url

NameError: name 'url' is not defined

In [21]:
page = '1'
url = base_url.format(page = page)

In [77]:
item.findAll(href=re.compile("pp1/s/.*/re_.*/"))[0].get('href')

'/pp1/s/tokyo/nerima-ku/re_2956389ruc/'

In [71]:
for i in item.findAll(href=re.compile("pp1/s/.*/re_.*/")):
    print(i.get('href'))

/pp1/s/tokyo/nerima-ku/re_2956389ruc/
/pp1/s/tokyo/kita-ku/re_29563791hs/
/pp1/s/tokyo/meguro-ku/re_2956378uq5/
/pp1/s/tokyo/chiyoda-ku/re_2956377fp7/
/pp1/s/tokyo/kita-ku/re_2956348ghm/
/pp1/s/tokyo/setagaya-ku/re_2956335jef/
/pp1/s/tokyo/nerima-ku/re_2956324suu/
/pp1/s/tokyo/minato-ku/re_2956288axf/
/pp1/s/tokyo/minato-ku/re_2956286vjc/
/pp1/s/tokyo/toshima-ku/re_2956285z2p/
/pp1/s/tokyo/taito-ku/re_2956284y7a/
/pp1/s/tokyo/sumida-ku/re_2956272jzh/
/pp1/s/tokyo/itabashi-ku/re_2956271boh/
/pp1/s/tokyo/ota-ku/re_2956269msu/
/pp1/s/tokyo/ota-ku/re_2956265rjy/
/pp1/s/tokyo/suginami-ku/re_2956263sgq/
/pp1/s/tokyo/shibuya-ku/re_2956262jwv/
/pp1/s/tokyo/bunkyo-ku/re_2956261us0/
/pp1/s/tokyo/shinjuku-ku/re_29562602us/
/pp1/s/tokyo/itabashi-ku/re_2956259izq/
/pp1/s/tokyo/ota-ku/re_2956258mpx/
/pp1/s/tokyo/bunkyo-ku/re_2956249uek/
/pp1/s/tokyo/shinjuku-ku/re_2956240xjo/
/pp1/s/tokyo/nerima-ku/re_2956232un7/
/pp1/s/tokyo/minato-ku/re_2956231xzo/
/pp1/s/tokyo/itabashi-ku/re_29562303yx/
/pp1/s/to

In [57]:
text = item.findAll(href=re.compile("pp1/s/.*/re_.*/"))[0].get('href')
# get('href')
text

'/pp1/s/tokyo/nerima-ku/re_2956389ruc/'

In [54]:
text

[<a href="/pp1/s/tokyo/nerima-ku/re_2956389ruc/" target="_blank">
 <ul class="prop_block">
 <li class="photo">
 <input data-closed_flg="N" id="ck_1" name="ck_pp" type="checkbox" value="2956389ruc"/><label for="ck_1">
 <img alt="不動産投資の投資用マンション" class="cate_icon" loading="lazy" src="/images/prop/cate_ONE.svg"/>
 <p><img alt="東京都練馬区の投資用マンション／豊島園_画像" loading="lazy" src="/upload/p2956/2956389/1661218999700-haj_wm_3229_000.jpg"/></p>
 </label>
 </li>
 <li class="main">
 <ul>
 <li><h3>オメガエステートよりお届け！「練馬」駅 徒歩3分！！</h3></li>
 <li>東京都練馬区豊玉北4-14-8</li>
 <li>都営大江戸線 豊島園駅 歩3分</li>
 <li><span class="new">New</span></li>
 </ul>
 </li>
 <li class="price">
 <ul>
 <li><span>730</span>万円</li>
 <li><span>7<span>.72</span></span>％</li>
 </ul>
 </li>
 <li>
 <ul>
 <li>専:15.00m²</li>
 </ul>
 </li>
 <li>
 <ul>
 <li>1985年1月</li>
 <li>1階/4階建</li>
 </ul>
 </li>
 </ul>
 </a>,
 <a href="/pp1/s/tokyo/kita-ku/re_29563791hs/" target="_blank">
 <ul class="prop_block">
 <li class="photo">
 <input data-closed_flg="N" id="ck